In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [33]:
import pandas as pd
import logging
from fast_bert.data_lm import BertLMDataBunch
from fast_bert.learner_lm import BertLMLearner
from pathlib import Path
from box import Box
import torch

In [4]:
fast_topics = pd.read_csv('/Users/jpnelson/2020/sul-dlss/ai-etd/data/topic_uri_label_utf8.csv', names=['URI', 'Label'])

In [41]:
args = Box({
    "seed": 42,
    "task_name": "etd_fulltext_lm",
    "model_name": "roberta-base",
    "model_type": "roberta",
    "train_batch_size": 16,
    "learning_rate": 4e-5,
    "num_train_epochs": 20,
    "fp16": False,
    "fp16_opt_level": "O2",
    "warmup_steps": 1000,
    "logging_steps": 0,
    "max_seq_length": 512,
    "multi_gpu": False
})
texts = []
for path in DATA_PATH.iterdir():
    texts.append(path.name)
logger = logging.getLogger()

In [8]:
DATA_PATH = Path("../tmp/etds/")
LOG_PATH = Path("logs")
MODEL_PATH = Path(f"lm_models_{args.model_type}")

DATA_PATH.mkdir(exist_ok=True)
MODEL_PATH.mkdir(exist_ok=True)
LOG_PATH.mkdir(exist_ok=True)

In [42]:
etd_databunch_lm = BertLMDataBunch.from_raw_corpus(
    data_dir=DATA_PATH,
    text_list=texts,
    tokenizer=args.model_name,
    batch_size_per_gpu=args.train_batch_size,
    multi_gpu=args.multi_gpu,
    model_type=args.model_type,
    logger=logger)

Setting 'max_len_single_sentence' is now deprecated. This value is automatically set up.


In [46]:
learner = BertLMLearner.from_pretrained_model(
    dataBunch=etd_databunch_lm,
    pretrained_path=args.model_name,
    output_dir=MODEL_PATH,
    metrics=[],
    device=torch.device('cpu'),
    logger=logger,
    multi_gpu=False,
    logging_steps=args.logging_steps,
    is_fp16=False)

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
torch.device("cpu")

device(type='cpu')

In [47]:
learner.fit(epochs=6,
            lr=6e-5,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="lamb")

/Users/jpnelson/2020/py3-env/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


(36, 3.590120461252001)

In [48]:
learner.save_model()

In [51]:
help(learner)

Help on BertLMLearner in module fast_bert.learner_lm object:

class BertLMLearner(fast_bert.learner_util.Learner)
 |  BertLMLearner(data: fast_bert.data_lm.BertLMDataBunch, model: torch.nn.modules.module.Module, pretrained_model_path, output_dir, metrics, device, logger, multi_gpu=True, is_fp16=True, warmup_steps=0, fp16_opt_level='O1', grad_accumulation_steps=1, max_grad_norm=1.0, adam_epsilon=1e-08, logging_steps=100)
 |  
 |  Method resolution order:
 |      BertLMLearner
 |      fast_bert.learner_util.Learner
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, data: fast_bert.data_lm.BertLMDataBunch, model: torch.nn.modules.module.Module, pretrained_model_path, output_dir, metrics, device, logger, multi_gpu=True, is_fp16=True, warmup_steps=0, fp16_opt_level='O1', grad_accumulation_steps=1, max_grad_norm=1.0, adam_epsilon=1e-08, logging_steps=100)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  fit(self, epochs, lr, validate

In [52]:
from fast_bert.prediction import BertClassificationPredictor

In [55]:
predicator = BertClassificationPredictor(
    model_path=MODEL_PATH/'model_out',
    label_path=Path('data/topic_uri_label_utf8.csv'),
    multi_label=True,
    model_type='roberta',
    do_lower_case=False)

AttributeError: 'PosixPath' object has no attribute 'decode'